In [ ]:
import requests
from lxml import html

def check_url(url, xpath):
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "max-age=0",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "If-Modified-Since": "Fri, 22 Nov 2024 18:10:33 GMT",
    }
    cookies = {
        "gcl_au": "1.1.581871697.1732298393",
        "useinternal": "true",
        "_gid": "GA1.2.2141075332.1732298393",
        # Ek çerezleri buraya ekleyin
    }

    response = requests.get(url, headers=headers, cookies=cookies)
    print(f"Status Code: {response.status_code}")

    # HTML'i lxml ile işleme
    tree = html.fromstring(response.content)

    # XPath'i doğrudan kullanma
    try:
        result = tree.xpath(xpath)
        if result:
            # Eğer XPath bir liste dönerse, ilk öğeyi döndür
            return result[0].text if hasattr(result[0], 'text') else str(result[0])
        else:
            return "XPath sonucu bulunamadı!"
    except Exception as e:
        return f"Hata: {str(e)}"


In [6]:
import requests
from lxml import html

In [7]:
def get_response_from_url(url):
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "max-age=0",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "If-Modified-Since": "Fri, 22 Nov 2024 18:10:33 GMT",
    }
    cookies = {
        "gcl_au": "1.1.581871697.1732298393",
        "useinternal": "true",
        "_gid": "GA1.2.2141075332.1732298393",
        # Ek çerezleri buraya ekleyin
    }

    response = requests.get(url, headers=headers, cookies=cookies)
    print(f"Status Code: {response.status_code}")
    return response.content

In [3]:
main_urls = ["https://www.hepsiburada.com/magaza/hepsiburada?kategori=2147483642_371965&tab=allproducts",
             "https://www.hepsiburada.com/magaza/kolaysepet?kategori=2147483638&tab=allproducts"]

In [4]:
one_page_product_count = 36
page_count = 10

In [8]:
for main_url in main_urls:
    for page in range(page_count):
        page += 1
        if page == 1:
            page_link = ""
        else:
            page_link = f"&sayfa={page}"
        response_content = get_response_from_url(url=main_url+page_link)
        tree = html.fromstring(response_content)

        for i in range(one_page_product_count):
            i += 1
            print(i)
            general_product_link_xpath = f"/html/body/div[2]/div/div/main/div/div/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[4]/div/div[2]/div/div/div/div/div/div/ul/li[{i}]/div/a"
            general_product_price_xpath = f"/html/body/div[2]/div/div/main/div/div/div/div[2]/div/div[2]/div[2]/div/div/div[2]/div[4]/div/div[2]/div/div/div/div/div/div/ul/li[{i}]/div/a/div[2]/div[4]"
            general_product_link = tree.xpath(f"{general_product_link_xpath}/@href")
            general_product_price = tree.xpath(general_product_price_xpath)

            if general_product_link and general_product_price:
                l = general_product_link[0].text if hasattr(general_product_link[0], 'text') else str(general_product_link[0])
                p = general_product_price[0].text if hasattr(general_product_price[0], 'text') else str(general_product_price[0])
                print(l, p)
    

Status Code: 200
1
/iphone-13-128-gb-beyaz-p-HBCV00000ODHHO?magaza=Hepsiburada 34.199,00
2
/xiaomi-redmi-note-13-pro-512gb-12gb-xiaomi-turkiye-garantili-siyah-p-HBCV00005PZUWF?magaza=Hepsiburada 15.999,00
3
/samsung-galaxy-m34-5g-6-gb-128-gb-samsung-turkiye-garantili-gumus-128-gb-gumus-p-HBCV00004WO01C?magaza=Hepsiburada 8.999,00
4
/xiaomi-redmi-note-13-pro-256gb-8gb-ram-xiaomi-turkiye-garantili-siyah-p-HBCV00005PZUWA?magaza=Hepsiburada 13.799,00
5
/iphone-13-128-gb-siyah-p-HBCV00000ODHHF?magaza=Hepsiburada 34.599,00
6
/iphone-16-pro-128gb-col-titanyum-p-HBCV00006Y4HGL?magaza=Hepsiburada 82.999,00
7
/iphone-16-pro-max-256gb-siyah-p-HBCV00006Y4HGT?magaza=Hepsiburada 99.999,00
8
/iphone-16-pro-max-256gb-naturel-titanyum-p-HBCV00006Y4HC7?magaza=Hepsiburada 99.899,00
9
/iphone-16-pro-max-256gb-col-titanyum-p-HBCV00006Y4HC5?magaza=Hepsiburada 99.999,00
10
/iphone-16-256gb-beyaz-p-HBCV00006Y4HFT?magaza=Hepsiburada 64.296,49
11
/iphone-15-128-gb-siyah-p-HBCV00004X9ZCH?magaza=Hepsiburada 51.84